In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [2]:
# pip install pypdf

In [3]:
import os
os.getcwd()

'c:\\Users\\66738\\OneDrive - Bain\\Documents\\Personal Docs\\Agents - AI\\RAG_\\Personal_projects'

In [4]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: sample.pdf
  ✓ Loaded 2 pages

Total documents loaded: 2


In [5]:
all_pdf_documents

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-25T10:57:01+05:30', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_enabled': 'true', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_setdate': '2025-02-19T16:19:54Z', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_method': 'Standard', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_name': 'defa4170-0d19-0005-0004-bc88714345d2', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_siteid': '4bfc8473-b4f8-4a68-83c6-6b4fc5438261', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_actionid': '290907e1-96ba-4391-8e68-48c3808eb77c', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_contentbits': '0', 'author': 'Aditya Gaurav', 'moddate': '2025-06-25T10:57:01+05:30', 'source': '..\\data\\sample.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'sample.pdf', 'file_type': 'pdf'}, page_content='WORK EXPERIENCE \n \nSr. Data Scientist | S

'''chunk_size means “the maximum allowed size of a chunk, measured by whatever your length_function counts.”

If length_function = len → count characters → chunk_size=500 = 500 characters.

If length_function = tokenizer → count tokens → chunk_size=500 = 500 tokens.'''

In [6]:
'''Suppose you have a 4,000-character PDF page.
With chunk_size=1000 and chunk_overlap=200:

Chunk 1: characters 0–1000

Chunk 2: characters 800–1800

Chunk 3: characters 1600–2600

Chunk 4: characters 2400–3400

Chunk 5: characters 3200–4000

✅ Overlap ensures continuity. If a sentence starts at the end of one chunk, it won’t get lost'''

'Suppose you have a 4,000-character PDF page.\nWith chunk_size=1000 and chunk_overlap=200:\n\nChunk 1: characters 0–1000\n\nChunk 2: characters 800–1800\n\nChunk 3: characters 1600–2600\n\nChunk 4: characters 2400–3400\n\nChunk 5: characters 3200–4000\n\n✅ Overlap ensures continuity. If a sentence starts at the end of one chunk, it won’t get lost'

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance.
    
    Splitting priority:
    1. Double newlines (\n\n) → paragraphs
    2. Single newlines (\n) → sentences/line breaks
    3. Spaces (" ") → words
    4. Characters ("") → fallback
    """
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print("\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [8]:
chunks=split_documents(all_pdf_documents)
chunks

Split 2 documents into 12 chunks

Example chunk:
Content: WORK EXPERIENCE 
 
Sr. Data Scientist | SymphonyAI | Dec’ 23-Ongoing 
 
➢  AI for Work : Developed Agentic AI Studio, a low -code/no-code, easily 
customizable SaaS product aimed at improving employee...
Metadata: {'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-25T10:57:01+05:30', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_enabled': 'true', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_setdate': '2025-02-19T16:19:54Z', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_method': 'Standard', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_name': 'defa4170-0d19-0005-0004-bc88714345d2', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_siteid': '4bfc8473-b4f8-4a68-83c6-6b4fc5438261', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_actionid': '290907e1-96ba-4391-8e68-48c3808eb77c', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_contentbits': '0', 'au

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-25T10:57:01+05:30', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_enabled': 'true', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_setdate': '2025-02-19T16:19:54Z', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_method': 'Standard', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_name': 'defa4170-0d19-0005-0004-bc88714345d2', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_siteid': '4bfc8473-b4f8-4a68-83c6-6b4fc5438261', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_actionid': '290907e1-96ba-4391-8e68-48c3808eb77c', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_contentbits': '0', 'author': 'Aditya Gaurav', 'moddate': '2025-06-25T10:57:01+05:30', 'source': '..\\data\\sample.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'sample.pdf', 'file_type': 'pdf'}, page_content='WORK EXPERIENCE \n \nSr. Data Scientist | S

In [9]:
# pip install -U sentence-transformers
# pip install chromadb


In [10]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\66738\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
class EmbedingManager():
    def __init__(self, model_name: str = "all-mpnet-base-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        try:
            self.model = SentenceTransformer(f"sentence-transformers/{self.model_name}")
            print('model loaded successfully')
        except Exception as e:
            print(f'The model failed to load {self.model_name}')

    def create_embedings(self, texts: List[str]) -> np.array:
        if not self.model:
            raise ('model is empty')
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
    
embeding_manager = EmbedingManager()

No sentence-transformers model found with name sentence-transformers/all-mpnet-base-v2. Creating a new one with mean pooling.


The model failed to load all-mpnet-base-v2
